Vector autoregressive model
===

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

Generate data from stable three dimensional VAR(1) process
---

In [ ]:
# parameters
# this example comes from Lutkepohl, "New Introduction fo Multiple Time Series Analysis"

d = 3
mu = np.zeros(d)
beta_true = np.array([[0.5, 0., 0.],[0.1, 0.1, 0.3],[0., 0.2, 0.3]])

noise_obs = 0.1
T = 128

In [ ]:
# generate synthetic data from true model
x_true = np.random.randn(T,d)*noise_obs
for t in xrange(1, T):
    x_true[t] += beta_true.dot(x_true[t-1])

In [ ]:
plt.plot(x_true);

Edward model
---

In [ ]:
from edward.models import Normal, InverseGamma, PointMass

mu = Normal(mu=tf.zeros([d]), sigma=10.*tf.ones([d]))
beta = Normal(mu=tf.zeros([d, d]), sigma=2.*tf.ones([d,d]))

noise_proc = tf.constant(0.1)  # InverseGamma(alpha=1.0, beta=1.0)
noise_obs = tf.constant(0.1)  # InverseGamma(alpha=1.0, beta=1.0)

x = [0] * T
x[0] = Normal(mu=mu, sigma=10.*tf.ones([d]))
for n in xrange(1, T):
    x[n] = Normal(mu=mu + tf.tensordot(beta, x[n-1], axes=[[1],[0]]),
                  sigma=noise_proc*tf.ones([d]))

MAP inference
---

In [ ]:
from itertools import izip

print("setting up distributions")
qmu = PointMass(params=tf.Variable(tf.zeros([d])))
qbeta = PointMass(params=tf.Variable(tf.zeros([d,d])))
print("constructing inference object")
inference = ed.MAP({beta: qbeta, mu: qmu}, data={xt: xt_true for xt, xt_true in izip(x, x_true)})
print("running inference")
inference.run()

print("parameter estimates:")
print("mu: ", qmu.value().eval())
print("beta:\n", qbeta.value().eval())

Variational inference
---

In [ ]:
print("setting up variational distributions")
qmu = Normal(mu=tf.Variable(tf.random_normal([d])), sigma=tf.nn.softplus(tf.Variable(tf.random_normal([d]))))
qbeta = Normal(mu=tf.Variable(tf.random_normal([d, d])), sigma=tf.nn.softplus(tf.Variable(tf.random_normal([d,d]))))
print("constructing inference object")
inference_vb = ed.KLqp({beta: qbeta, mu: qmu}, data={xt: xt_true for xt, xt_true in zip(x, x_true)})
print("running VB inference")
inference_vb.run()

print("parameter estimates:")
print("mu: ", qmu.mean().eval())
print("beta:\n", qbeta.mean().eval())

Compare with `statsmodels`
---

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

var1 = VAR(x_true)
res = var1.fit(1, ic=None, trend='c')

In [ ]:
mu_sm = res.params[0]
beta_sm = res.params[1:].T
print("mu_sm: ", mu_sm)
print("beta_sm:\n", beta_sm)